# Master Script 7: Calculate bootstrapping bias-corrected cross-validation (BBC-CV) area under the receiver operating characteristic curve (AUC) and classification metrics

Shubhayu Bhattacharyay
<br>
University of Cambridge
<br>
Johns Hopkins University
<br>
email address: sb2406@cam.ac.uk

## Contents:
### I. Initialization
### II. Define functions for calculating metrics and ROC curve with BBC-CV
### III. Calculate metrics for GCSm threshold-level detection
### IV. Calculate metrics for GOSE (at discharge) threshold-level prediction
### V. Calculate metrics for GOSE (at 12 months) threshold-level prediction

## I. Initialization

### Import necessary packages and methods

In [1]:
# Fundamental methods
import os
import re
import sys
import time
import glob
import random
import warnings
import itertools
import numpy as np
import pandas as pd
import pickle as cp
import seaborn as sns
import multiprocessing
from scipy import interp
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import clear_output

# tqdm method for progress monitoring
from tqdm import tqdm

# SciKit-Learn methods
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc, mean_squared_error, accuracy_score, confusion_matrix
from sklearn.calibration import calibration_curve

### Establish necessary parameters

In [2]:
# Establish number of cores for parallel processing (default: save 2 cores)
NUM_CORES = multiprocessing.cpu_count() - 2

# Establish number of resamples for bias-corrected bootstrapping of confidence intervals
NUM_BOOTSTRAPS = 1000

# Ignore potential warnings
warnings.filterwarnings(action='ignore')

## II. Define functions for calculating metrics and ROC curve with BBC-CV

### Function for calculating AUC and ROC axes

In [3]:
def calculate_bs_AUC(
    curr_compiled_predictions,
    curr_in_sample_UPIs,
    metric='AUC',
    roc_axes=True,
    out_interp_fpr=np.linspace(0, 1, num=200),
    ):

    # Derive list of unique UPIs based on compiled prediction dataframe

    uniq_UPIs = curr_compiled_predictions.UPI.unique()

    # Get current out samples based on current bootstrap in-sample

    curr_out_sample_UPIs = np.setdiff1d(uniq_UPIs, curr_in_sample_UPIs)

    # Separate predictions into in-sample and out-sample based on current bootstrap resample

    curr_in_sample_predictions = \
        curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_in_sample_UPIs)]
    curr_out_sample_predictions = \
        curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_out_sample_UPIs)]

    # Calculate in-sample AUCs to determine optimal model configuration

    grouped_in_sample_predictions = \
        curr_in_sample_predictions.groupby('ConfigIdx', sort=False)
    in_sample_AUCs = pd.DataFrame(np.empty((0, 2)), columns=['ConfigIdx'
                                  , 'AUC'])

    # Iterate through predictions group by configuration index and calculate group-specific AUC

    for (name, group) in grouped_in_sample_predictions:
        in_sample_AUCs = \
            in_sample_AUCs.append(pd.DataFrame({'ConfigIdx': name,
                                  'AUC': roc_auc_score(group.TrueLabel,
                                  group.Prob)}, index=[0]),
                                  ignore_index=True)

    # Fix datatype of `ConfigIdx`

    in_sample_AUCs.ConfigIdx = in_sample_AUCs.ConfigIdx.astype('int')

    # Isolate optimal configuration index for current sampling

    curr_opt_ConfigIdx = in_sample_AUCs.nlargest(1, 'AUC', keep='first'
            ).ConfigIdx.values[0]

    # Filter out out-sample predictions of optimal configuration index

    curr_out_sample_predictions = \
        curr_out_sample_predictions[curr_out_sample_predictions.ConfigIdx
                                    == curr_opt_ConfigIdx]

    # Calculate out-sample AUC

    out_sample_AUC = \
        roc_auc_score(curr_out_sample_predictions.TrueLabel,
                      curr_out_sample_predictions.Prob)

    # Calculate out-sample ROC curve axes and interpolate to common FPR axis

    if roc_axes:
        (out_sample_fpr, out_sample_tpr, _) = \
            roc_curve(curr_out_sample_predictions.TrueLabel,
                      curr_out_sample_predictions.Prob)
        out_interp_tpr = np.interp(out_interp_fpr, out_sample_fpr,
                                   out_sample_tpr)

        # Return AUC, optimal configuration, and ROC curve axes

        return (out_sample_AUC, curr_opt_ConfigIdx, out_interp_fpr,
                out_interp_tpr)
    else:

        # Return just AUC and optimal configuration if ROC not requested

        return (out_sample_AUC, curr_opt_ConfigIdx)

### Function for calculating classification metrics
Possible metrics: `precision`, `recall` (i.e., sensitivity), `f1-score`, and `specificity`

In [4]:
def calculate_bs_classification(curr_compiled_predictions,
                                curr_in_sample_UPIs, metric='precision'
                                ):

    # Derive list of unique UPIs based on compiled prediction dataframe

    uniq_UPIs = curr_compiled_predictions.UPI.unique()

    # Get current out samples based on current bootstrap in-sample

    curr_out_sample_UPIs = np.setdiff1d(uniq_UPIs, curr_in_sample_UPIs)

    # Separate predictions into in-sample and out-sample based on current bootstrap resample

    curr_in_sample_predictions = \
        curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_in_sample_UPIs)]
    curr_out_sample_predictions = \
        curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_out_sample_UPIs)]

    # Calculate in-sample metrics to determine optimal model configuration

    grouped_in_sample_predictions = \
        curr_in_sample_predictions.groupby('ConfigIdx', sort=False)
    in_sample_classifications = pd.DataFrame(np.empty((0, 2)),
            columns=['ConfigIdx', 'Metric'])

    # Iterate through predictions group by configuration index and calculate group-specific metric

    for (name, group) in grouped_in_sample_predictions:
        if metric == 'specificity':
            curr_group_metric_value = \
                classification_report(group.TrueLabel, (group.Prob
                    >= .5).astype('int').values, output_dict=True)['0'
                    ]['recall']
        else:
            curr_group_metric_value = \
                classification_report(group.TrueLabel, (group.Prob
                    >= .5).astype('int').values, output_dict=True)['1'
                    ][metric]
        in_sample_classifications = \
            in_sample_classifications.append(pd.DataFrame({'ConfigIdx': name,
                'Metric': curr_group_metric_value}, index=[0]),
                ignore_index=True)

    # Fix datatype of `ConfigIdx`

    in_sample_classifications.ConfigIdx = \
        in_sample_classifications.ConfigIdx.astype('int')

    # Isolate optimal configuration index for current sampling

    curr_opt_ConfigIdx = in_sample_classifications.nlargest(1, 'Metric'
            , keep='first').ConfigIdx.values[0]

    # Filter out out-sample predictions of optimal configuration index

    curr_out_sample_predictions = \
        curr_out_sample_predictions[curr_out_sample_predictions.ConfigIdx
                                    == curr_opt_ConfigIdx]

    # Calculate out-sample metric

    if metric == 'specificity':
        out_sample_classifications = \
            classification_report(curr_out_sample_predictions.TrueLabel,
                                  (curr_out_sample_predictions.Prob
                                  >= .5).astype('int').values,
                                  output_dict=True)['0']['recall']
    else:
        out_sample_classifications = \
            classification_report(curr_out_sample_predictions.TrueLabel,
                                  (curr_out_sample_predictions.Prob
                                  >= .5).astype('int').values,
                                  output_dict=True)['1'][metric]

    # Return out-sample metric value and optimal configuration

    return (out_sample_classifications, curr_opt_ConfigIdx)

### Sub-function for each parallelization core

In [5]:
def _calculate_bs_metrics(
    metric_fun,
    curr_compiled_predictions,
    size,
    metric,
    progress_bar=True,
    progress_bar_desc='',
    ):

    # Create random generator instance

    rg = np.random.default_rng()

    # Get list of unique UPIs

    uniq_UPIs = curr_compiled_predictions.UPI.unique()

    # Derive current bootstrapping sample

    bootstrap_resamples = rg.choice(uniq_UPIs, (len(uniq_UPIs), size),
                                    replace=True)

    # Go through current resamples and ensure positive and negative cases exist for each resample

    for bs_col in range(bootstrap_resamples.shape[1]):

        # Get current in and out samples based on bootstrap resample

        curr_in_sample_UPIs = np.unique(bootstrap_resamples[:, bs_col])
        curr_out_sample_UPIs = np.setdiff1d(uniq_UPIs,
                curr_in_sample_UPIs)

        # Split current compiled prediction dataframe into in-sample and out-sample predictions

        curr_in_sample_predictions = \
            curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_in_sample_UPIs)]
        curr_out_sample_predictions = \
            curr_compiled_predictions[curr_compiled_predictions.UPI.isin(curr_out_sample_UPIs)]

        # If either the in or out sample contains only one outcome, resample current bootstrap until there is

        if (len(curr_in_sample_predictions.TrueLabel.unique()) == 1) \
            | (len(curr_out_sample_predictions.TrueLabel.unique())
               == 1):

            fail_condition = True

            while fail_condition:
                temp_resample = rg.choice(uniq_UPIs, (len(uniq_UPIs),
                        1), replace=True)
                temp_in_sample_UPIs = np.unique(temp_resample[:, 0])
                temp_out_sample_UPIs = np.setdiff1d(uniq_UPIs,
                        temp_in_sample_UPIs)
                temp_in_sample_predictions = \
                    curr_compiled_predictions[curr_compiled_predictions.UPI.isin(temp_in_sample_UPIs)]
                temp_out_sample_predictions = \
                    curr_compiled_predictions[curr_compiled_predictions.UPI.isin(temp_out_sample_UPIs)]

                if (len(temp_in_sample_predictions.TrueLabel.unique())
                    == 2) \
                    & (len(temp_out_sample_predictions.TrueLabel.unique())
                       == 2):
                    bootstrap_resamples[:, bs_col] = temp_resample[:, 0]
                    fail_condition = False

    if progress_bar:
        iterator = tqdm(range(size), desc=progress_bar_desc)
    else:
        iterator = range(size)

    return [metric_fun(curr_compiled_predictions,
            curr_in_sample_UPIs=np.unique(bootstrap_resamples[:,
            bs_idx]), metric=metric) for bs_idx in iterator]

### Surface function for BBC-CV metric calculation with parallel processing

In [6]:
def start_calculate_bs_metrics(
    metric_fun,
    curr_compiled_predictions,
    num_bs,
    n_cores,
    metric='AUC',
    progress_bar=True,
    progress_bar_desc='',
    ):

    # Establish sizes of bootstrap replicates for each core

    sizes = [num_bs // n_cores for _ in range(n_cores)]
    sizes[-1] += num_bs - sum(sizes)

    # Build arguments for metric sub-functions

    arg_iterable = [(
        metric_fun,
        curr_compiled_predictions,
        s,
        metric,
        progress_bar,
        progress_bar_desc,
        ) for s in sizes]

    # Run metric sub-function in parallel

    with multiprocessing.Pool(n_cores) as pool:
        result = pool.starmap(_calculate_bs_metrics, arg_iterable)

    return np.concatenate(result)

## III. Calculate metrics for GCSm threshold-level detection

In [8]:
# Extract directories of different observation windows

obs_window_dirs = \
    glob.glob('../results/GCSm_threshold_prediction/*_h_obs_window/')

# Iterate through observation window lengths

for curr_obs_window_dir in obs_window_dirs[::-1]:

    # Extract current observation window length from directory name

    curr_obs_window_length = \
        float(re.search('GCSm_threshold_prediction/(.*)_h_obs_window/'
              , curr_obs_window_dir).group(1))

    # Status update on current observation window

    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours started.')

    # Extract compiled prediction file names in current observation window

    thresh_pred_files = glob.glob(os.path.join(curr_obs_window_dir,
                                  '*_compiled_predictions.csv'))

    # Iterate through different threshold values

    for curr_tresh_pred_file in thresh_pred_files:

        # Extract current GCSm threshold from the file name

        curr_GCSm_thresh = \
            re.search('_h_obs_window/(.*)_compiled_predictions.csv',
                      curr_tresh_pred_file).group(1)

        # Status update on current observation window

        print('GCSm Threshold: ' + curr_GCSm_thresh + ' started.')

        # Load current compiled prediction file for threshold

        curr_compiled_predictions = pd.read_csv(curr_tresh_pred_file)

        # Skip current GCSm threshold and observation window combination if no predictions are available

        if curr_compiled_predictions.shape[0] == 0:
            print('GCSm Threshold: ' + curr_GCSm_thresh
                + ' skipped due to no predictions.')
            continue

        # Create new indicator for positive and negative cases

        curr_compiled_predictions['PosCases'] = \
            curr_compiled_predictions.TrueLabel
        curr_compiled_predictions['NegCases'] = \
            (curr_compiled_predictions.TrueLabel == 0).astype('int')

        # Group predictions by UPI and calculate number of positive and negative cases by group

        case_distribution = \
            pd.concat([curr_compiled_predictions.groupby('UPI'
                      )['PosCases'].sum(),
                      curr_compiled_predictions.groupby('UPI'
                      )['NegCases'].sum()], axis=1)

        # If there are not 2 or more patients for each case, skip current GCSm threshold and observation window combination

        if ((case_distribution.PosCases >= 1).sum() < 2) \
            | ((case_distribution.NegCases >= 1).sum() < 2):
            print('GCSm Threshold: ' + curr_GCSm_thresh
                + ' skipped due to insufficient case distribution.')
            continue

        # Calculate AUC and ROC axes

        bs_AUC_output = start_calculate_bs_metrics(
            calculate_bs_AUC,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='AUC',
            progress_bar=True,
            progress_bar_desc=curr_GCSm_thresh + ' AUC bootstrapping',
            )
        bs_AUC_df = pd.DataFrame(bs_AUC_output[:, 0:2],
                                 columns=['Values', 'ConfigIdx'])
        bs_AUC_df['Metrics'] = 'AUC'

        # Calculate precision

        bs_precision_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='precision',
            progress_bar=True,
            progress_bar_desc=curr_GCSm_thresh
                + ' precision bootstrapping',
            )
        bs_precision_df = pd.DataFrame(bs_precision_output,
                columns=['Values', 'ConfigIdx'])
        bs_precision_df['Metrics'] = 'precision'

        # Calculate recall

        bs_recall_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='recall',
            progress_bar=True,
            progress_bar_desc=curr_GCSm_thresh + ' recall bootstrapping'
                ,
            )
        bs_recall_df = pd.DataFrame(bs_recall_output, columns=['Values'
                                    , 'ConfigIdx'])
        bs_recall_df['Metrics'] = 'recall'

        # Calculate f1-score

        bs_f1_score_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='f1-score',
            progress_bar=True,
            progress_bar_desc=curr_GCSm_thresh
                + ' f1-score bootstrapping',
            )
        bs_f1_score_df = pd.DataFrame(bs_f1_score_output,
                columns=['Values', 'ConfigIdx'])
        bs_f1_score_df['Metrics'] = 'f1_score'

        # Calculate specificity

        bs_specificity_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='specificity',
            progress_bar=True,
            progress_bar_desc=curr_GCSm_thresh
                + ' specificity bootstrapping',
            )
        bs_specificity_df = pd.DataFrame(bs_specificity_output,
                columns=['Values', 'ConfigIdx'])
        bs_specificity_df['Metrics'] = 'specificity'

        # Compile out-sample metrics into single dataframe

        compiled_out_sample_metrics = pd.concat([bs_AUC_df,
                bs_precision_df, bs_recall_df, bs_f1_score_df,
                bs_specificity_df], ignore_index=True)

        # Add metadata of current iteration to compiled dataframe

        compiled_out_sample_metrics['Threshold'] = curr_GCSm_thresh
        compiled_out_sample_metrics['ObsWindow'] = \
            curr_obs_window_length

        # Reorder columns to intended order for subsequent R processing

        compiled_out_sample_metrics = \
            compiled_out_sample_metrics[['ConfigIdx', 'Metrics',
                'Values', 'Threshold', 'ObsWindow']]

        # Save compiled metric dataframe in the correct observation window directory

        compiled_out_sample_metrics.to_csv(os.path.join(curr_obs_window_dir,
                curr_GCSm_thresh + '_compiled_metrics.csv'),
                index=False)

        # Extract ROC curve axis information and convert to dataframe

        compiled_out_sample_ROC = pd.DataFrame({
            'ConfigIdx': np.repeat(bs_AUC_output[:, 1], 200),
            'FPR': np.concatenate(bs_AUC_output[:, 2]),
            'TPR': np.concatenate(bs_AUC_output[:, 3]),
            'Threshold': curr_GCSm_thresh,
            'ObsWindow': curr_obs_window_length,
            })

        # Save the compiled ROC dataframe in the correct observation window directory

        compiled_out_sample_ROC.to_csv(os.path.join(curr_obs_window_dir,
                curr_GCSm_thresh + '_compiled_ROC.csv'), index=False)

        # Clear output and signify end of threshold

        clear_output()
        print('Observation window: ' + str(curr_obs_window_length)
            + ' hours started.')
        print('GCSm Threshold: ' + curr_GCSm_thresh + ' completed.')

    # Clear output and signify end of threshold

    clear_output()
    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours completed.')

Observation window: 0.05 hours completed.


## IV. Calculate metrics for GOSE (at discharge) threshold-level prediction

In [9]:
# Extract directories of different observation windows

obs_window_dirs = \
    glob.glob('../results/GOSE_threshold_prediction/*_h_obs_window/')

# Iterate through observation window lengths

for curr_obs_window_dir in obs_window_dirs[::-1]:

    # Extract current observation window length from directory name

    curr_obs_window_length = \
        float(re.search('GOSE_threshold_prediction/(.*)_h_obs_window/'
              , curr_obs_window_dir).group(1))

    # Status update on current observation window

    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours started.')

    # Extract compiled prediction file names in current observation window

    thresh_pred_files = glob.glob(os.path.join(curr_obs_window_dir,
                                  '*_compiled_predictions.csv'))

    # Iterate through different threshold values

    for curr_tresh_pred_file in thresh_pred_files:

        # Extract current GOSE threshold from the file name

        curr_GOSE_thresh = \
            re.search('_h_obs_window/(.*)_compiled_predictions.csv',
                      curr_tresh_pred_file).group(1)

        # Status update on current observation window

        print('GOSE Threshold: ' + curr_GOSE_thresh + ' started.')

        # Load current compiled prediction file for threshold

        curr_compiled_predictions = pd.read_csv(curr_tresh_pred_file)

        # Skip current GOSE threshold and observation window combination if no predictions are available

        if curr_compiled_predictions.shape[0] == 0:
            print('GOSE Threshold: ' + curr_GOSE_thresh
                + ' skipped due to no predictions.')
            continue

        # Create new indicator for positive and negative cases

        curr_compiled_predictions['PosCases'] = \
            curr_compiled_predictions.TrueLabel
        curr_compiled_predictions['NegCases'] = \
            (curr_compiled_predictions.TrueLabel == 0).astype('int')

        # Group predictions by UPI and calculate number of positive and negative cases by group

        case_distribution = \
            pd.concat([curr_compiled_predictions.groupby('UPI'
                      )['PosCases'].sum(),
                      curr_compiled_predictions.groupby('UPI'
                      )['NegCases'].sum()], axis=1)

        # If there are not 2 or more patients for each case, skip current GOSE threshold and observation window combination

        if ((case_distribution.PosCases >= 1).sum() < 2) \
            | ((case_distribution.NegCases >= 1).sum() < 2):
            print('GOSE Threshold: ' + curr_GOSE_thresh
                + ' skipped due to insufficient case distribution.')
            continue

        # Calculate AUC and ROC axes

        bs_AUC_output = start_calculate_bs_metrics(
            calculate_bs_AUC,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='AUC',
            progress_bar=True,
            progress_bar_desc=curr_GOSE_thresh + ' AUC bootstrapping',
            )
        bs_AUC_df = pd.DataFrame(bs_AUC_output[:, 0:2],
                                 columns=['Values', 'ConfigIdx'])
        bs_AUC_df['Metrics'] = 'AUC'

        # Calculate precision

        bs_precision_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='precision',
            progress_bar=True,
            progress_bar_desc=curr_GOSE_thresh
                + ' precision bootstrapping',
            )
        bs_precision_df = pd.DataFrame(bs_precision_output,
                columns=['Values', 'ConfigIdx'])
        bs_precision_df['Metrics'] = 'precision'

        # Calculate recall

        bs_recall_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='recall',
            progress_bar=True,
            progress_bar_desc=curr_GOSE_thresh + ' recall bootstrapping'
                ,
            )
        bs_recall_df = pd.DataFrame(bs_recall_output, columns=['Values'
                                    , 'ConfigIdx'])
        bs_recall_df['Metrics'] = 'recall'

        # Calculate f1-score

        bs_f1_score_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='f1-score',
            progress_bar=True,
            progress_bar_desc=curr_GOSE_thresh
                + ' f1-score bootstrapping',
            )
        bs_f1_score_df = pd.DataFrame(bs_f1_score_output,
                columns=['Values', 'ConfigIdx'])
        bs_f1_score_df['Metrics'] = 'f1_score'

        # Calculate specificity

        bs_specificity_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='specificity',
            progress_bar=True,
            progress_bar_desc=curr_GOSE_thresh
                + ' specificity bootstrapping',
            )
        bs_specificity_df = pd.DataFrame(bs_specificity_output,
                columns=['Values', 'ConfigIdx'])
        bs_specificity_df['Metrics'] = 'specificity'

        # Compile out-sample metrics into single dataframe

        compiled_out_sample_metrics = pd.concat([bs_AUC_df,
                bs_precision_df, bs_recall_df, bs_f1_score_df,
                bs_specificity_df], ignore_index=True)

        # Add metadata of current iteration to compiled dataframe

        compiled_out_sample_metrics['Threshold'] = curr_GOSE_thresh
        compiled_out_sample_metrics['ObsWindow'] = \
            curr_obs_window_length

        # Reorder columns to intended order for subsequent R processing

        compiled_out_sample_metrics = \
            compiled_out_sample_metrics[['ConfigIdx', 'Metrics',
                'Values', 'Threshold', 'ObsWindow']]

        # Save compiled metric dataframe in the correct observation window directory

        compiled_out_sample_metrics.to_csv(os.path.join(curr_obs_window_dir,
                curr_GOSE_thresh + '_compiled_metrics.csv'),
                index=False)

        # Extract ROC curve axis information and convert to dataframe

        compiled_out_sample_ROC = pd.DataFrame({
            'ConfigIdx': np.repeat(bs_AUC_output[:, 1], 200),
            'FPR': np.concatenate(bs_AUC_output[:, 2]),
            'TPR': np.concatenate(bs_AUC_output[:, 3]),
            'Threshold': curr_GOSE_thresh,
            'ObsWindow': curr_obs_window_length,
            })

        # Save the compiled ROC dataframe in the correct observation window directory

        compiled_out_sample_ROC.to_csv(os.path.join(curr_obs_window_dir,
                curr_GOSE_thresh + '_compiled_ROC.csv'), index=False)

        # Clear output and signify end of threshold

        clear_output()
        print('Observation window: ' + str(curr_obs_window_length)
            + ' hours started.')
        print('GOSE Threshold: ' + curr_GOSE_thresh + ' completed.')

    # Clear output and signify end of threshold

    clear_output()
    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours completed.')

Observation window: 0.05 hours completed.


## V. Calculate metrics for GOSE (at 12 months) threshold-level prediction

In [9]:
# Extract directories of different observation windows

obs_window_dirs = \
    glob.glob('../results/GOSE12m_threshold_prediction/*_h_obs_window/')

# Iterate through observation window lengths

for curr_obs_window_dir in obs_window_dirs:

    # Extract current observation window length from directory name

    curr_obs_window_length = \
        float(re.search('GOSE12m_threshold_prediction/(.*)_h_obs_window/'
              , curr_obs_window_dir).group(1))

    # Status update on current observation window

    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours started.')

    # Extract compiled prediction file names in current observation window

    thresh_pred_files = glob.glob(os.path.join(curr_obs_window_dir,
                                  '*_compiled_predictions.csv'))

    # Iterate through different threshold values

    for curr_tresh_pred_file in thresh_pred_files:

        # Extract current GOSE12m threshold from the file name

        curr_GOSE12m_thresh = \
            re.search('_h_obs_window/(.*)_compiled_predictions.csv',
                      curr_tresh_pred_file).group(1)

        # Status update on current observation window

        print('GOSE12m Threshold: ' + curr_GOSE12m_thresh + ' started.')

        # Load current compiled prediction file for threshold

        curr_compiled_predictions = pd.read_csv(curr_tresh_pred_file)

        # Skip current GOSE12m threshold and observation window combination if no predictions are available

        if curr_compiled_predictions.shape[0] == 0:
            print('GOSE12m Threshold: ' + curr_GOSE12m_thresh
                + ' skipped due to no predictions.')
            continue

        # Create new indicator for positive and negative cases

        curr_compiled_predictions['PosCases'] = \
            curr_compiled_predictions.TrueLabel
        curr_compiled_predictions['NegCases'] = \
            (curr_compiled_predictions.TrueLabel == 0).astype('int')

        # Group predictions by UPI and calculate number of positive and negative cases by group

        case_distribution = \
            pd.concat([curr_compiled_predictions.groupby('UPI'
                      )['PosCases'].sum(),
                      curr_compiled_predictions.groupby('UPI'
                      )['NegCases'].sum()], axis=1)

        # If there are not 2 or more patients for each case, skip current GOSE12m threshold and observation window combination

        if ((case_distribution.PosCases >= 1).sum() < 2) \
            | ((case_distribution.NegCases >= 1).sum() < 2):
            print('GOSE12m Threshold: ' + curr_GOSE12m_thresh
                + ' skipped due to insufficient case distribution.')
            continue

        # Calculate AUC and ROC axes

        bs_AUC_output = start_calculate_bs_metrics(
            calculate_bs_AUC,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='AUC',
            progress_bar=True,
            progress_bar_desc=curr_GOSE12m_thresh + ' AUC bootstrapping',
            )
        bs_AUC_df = pd.DataFrame(bs_AUC_output[:, 0:2],
                                 columns=['Values', 'ConfigIdx'])
        bs_AUC_df['Metrics'] = 'AUC'

        # Calculate precision

        bs_precision_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='precision',
            progress_bar=True,
            progress_bar_desc=curr_GOSE12m_thresh
                + ' precision bootstrapping',
            )
        bs_precision_df = pd.DataFrame(bs_precision_output,
                columns=['Values', 'ConfigIdx'])
        bs_precision_df['Metrics'] = 'precision'

        # Calculate recall

        bs_recall_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='recall',
            progress_bar=True,
            progress_bar_desc=curr_GOSE12m_thresh + ' recall bootstrapping'
                ,
            )
        bs_recall_df = pd.DataFrame(bs_recall_output, columns=['Values'
                                    , 'ConfigIdx'])
        bs_recall_df['Metrics'] = 'recall'

        # Calculate f1-score

        bs_f1_score_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='f1-score',
            progress_bar=True,
            progress_bar_desc=curr_GOSE12m_thresh
                + ' f1-score bootstrapping',
            )
        bs_f1_score_df = pd.DataFrame(bs_f1_score_output,
                columns=['Values', 'ConfigIdx'])
        bs_f1_score_df['Metrics'] = 'f1_score'

        # Calculate specificity

        bs_specificity_output = start_calculate_bs_metrics(
            calculate_bs_classification,
            curr_compiled_predictions,
            NUM_BOOTSTRAPS,
            NUM_CORES,
            metric='specificity',
            progress_bar=True,
            progress_bar_desc=curr_GOSE12m_thresh
                + ' specificity bootstrapping',
            )
        bs_specificity_df = pd.DataFrame(bs_specificity_output,
                columns=['Values', 'ConfigIdx'])
        bs_specificity_df['Metrics'] = 'specificity'

        # Compile out-sample metrics into single dataframe

        compiled_out_sample_metrics = pd.concat([bs_AUC_df,
                bs_precision_df, bs_recall_df, bs_f1_score_df,
                bs_specificity_df], ignore_index=True)

        # Add metadata of current iteration to compiled dataframe

        compiled_out_sample_metrics['Threshold'] = curr_GOSE12m_thresh
        compiled_out_sample_metrics['ObsWindow'] = \
            curr_obs_window_length

        # Reorder columns to intended order for subsequent R processing

        compiled_out_sample_metrics = \
            compiled_out_sample_metrics[['ConfigIdx', 'Metrics',
                'Values', 'Threshold', 'ObsWindow']]

        # Save compiled metric dataframe in the correct observation window directory

        compiled_out_sample_metrics.to_csv(os.path.join(curr_obs_window_dir,
                curr_GOSE12m_thresh + '_compiled_metrics.csv'),
                index=False)

        # Extract ROC curve axis information and convert to dataframe

        compiled_out_sample_ROC = pd.DataFrame({
            'ConfigIdx': np.repeat(bs_AUC_output[:, 1], 200),
            'FPR': np.concatenate(bs_AUC_output[:, 2]),
            'TPR': np.concatenate(bs_AUC_output[:, 3]),
            'Threshold': curr_GOSE12m_thresh,
            'ObsWindow': curr_obs_window_length,
            })

        # Save the compiled ROC dataframe in the correct observation window directory

        compiled_out_sample_ROC.to_csv(os.path.join(curr_obs_window_dir,
                curr_GOSE12m_thresh + '_compiled_ROC.csv'), index=False)

        # Clear output and signify end of threshold

        clear_output()
        print('Observation window: ' + str(curr_obs_window_length)
            + ' hours started.')
        print('GOSE12m Threshold: ' + curr_GOSE12m_thresh + ' completed.')

    # Clear output and signify end of threshold

    clear_output()
    print('Observation window: ' + str(curr_obs_window_length)
        + ' hours completed.')

Observation window: 1.0 hours completed.
